In [1]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
from mediapipe.tasks.python import vision
import threading
import cv2
import time

BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
HandLandmarkerResult = mp.tasks.vision.HandLandmarkerResult
VisionRunningMode = mp.tasks.vision.RunningMode

In [2]:
# 영상에서 손을 감지하고 결과를 처리하는 클래스

class HandLandmarkerAndResult:
    def __init__(self):
        base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
        self.options = vision.HandLandmarkerOptions(
            base_options=base_options,
            running_mode=vision.RunningMode.LIVE_STREAM,
            num_hands=2,
            result_callback=self.process_result
        )
        self.hand_landmarker = vision.HandLandmarker.create_from_options(self.options)
        self.result = None
        self.event = threading.Event()

    def process_result(self, result: vision.HandLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
        """감지 결과를 처리하는 콜백 함수"""
        self.result = result
        self.event.set()  # 결과가 준비되었음을 알림

    def detect_async(self, frame, timestamp_ms):
        """비동기적으로 손을 감지하고 결과를 self.result에 저장합니다."""

        # mediapipe Image 객체로 변환
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        # 비동기적으로 감지 실행
        self.hand_landmarker.detect_async(mp_image, timestamp_ms)

    def close(self):
        self.hand_landmarker.close()


In [3]:
# 손에 랜드마크 그리는 함수
MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

  return annotated_image

In [4]:

# HandLandmarker 객체 생성하기
detector = HandLandmarkerAndResult()

# 캠 불러오기.
cap = cv2.VideoCapture(0)  

while True:  # 카메라 동작하는 동안 계속하여 반복하기
    ret, frame = cap.read()  # 비디오 스트림(카메라 캡처)
    if not ret:  # 프레임을 가져오지 못하면 종료
        break
    frame = cv2.flip(frame, 1)  # 화면 좌우 반전

    # 비동기적으로 손 감지
    timestamp_ms = cv2.getTickCount() / cv2.getTickFrequency() * 1000  # 타임스탬프 생성
    detector.detect_async(frame, int(timestamp_ms))

    # 결과를 기다림
    detector.event.wait()

    # 결과 처리
    detection_result = detector.result

    # 손 감지 결과가 있으면 이미지에 그리기
    if detection_result:
        frame = draw_landmarks_on_image(frame, detection_result)
#         frame = put_handedness_on_image(frame, detection_result)
#         frame = put_index_finger_on_image(frame, detection_result)
#         frame = count_fingers_raised(frame, detection_result)
        cv2.imshow('frame', frame)
    else:
        # 손 감지 결과가 없으면 원본 이미지 출력
        cv2.imshow('frame', frame)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()  # 카메라 해제
cv2.destroyAllWindows()


In [10]:
# 왼손, 오른손 표기
MARGIN = 10  # pixels
SIZE = 1
THICKNESS = 2
COLOR = (88, 205, 54) # vibrant green

def put_handedness_on_image(rgb_image, detection_result):

    
    
    

    return rgb_image

In [ ]:
#왼손, 오른손 표기( 인덱스를 사용해서 풀기 )

def put_handedness_on_image(rgb_image, detection_result):

    
    
    return rgb_image

In [17]:
#두 번째 손가락 표기하기

def put_index_finger_on_image(rgb_image, detection_result):


    # Loop through the detected hands to visualize.
    for idx in range(len(hand_landmarks_list)):
    


    return rgb_image

In [ ]:
# 손의 가장 왼쪽 위에 왼손, 오른손 표기하기

def put_handedness_on_image(rgb_image, detection_result):

    
    
    return rgb_image

In [24]:
# 손가락이 몇 개 올라와 있는지 세는 함수

def count_fingers_raised(rgb_image, detection_result: mp.tasks.vision.HandLandmarkerResult):

    
    
    return rgb_image

